# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nago,26.5881,127.9761,27.47,83,45,6.31,JP,1696164298
1,1,port-aux-francais,-49.3500,70.2167,5.68,97,100,15.85,TF,1696163925
2,2,jamestown,42.0970,-79.2353,14.77,100,100,0.45,US,1696163934
3,3,richards bay,-28.7830,32.0377,18.79,64,100,5.20,ZA,1696164300
4,4,punta arenas,-53.1500,-70.9167,4.06,70,40,10.29,CL,1696163997


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
scatter_plot = city_data_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Humidity',  # Use humidity as color
    s='Humidity',  # Use humidity as size
    cmap='Blues',       # Specify the color map
    colorbar=True,      # Show the color bar
    title='City Humidity Map',
    width=800,
    height=600
)

# Display the map
scatter_plot

:Scatter   [Lng]   (Lat,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values


# Define the conditions for filtering
temperature_condition = (city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21)
wind_speed_condition = city_data_df['Wind Speed'] < 4.5
cloudiness_condition = city_data_df['Cloudiness'] == 0

# Combine the conditions using the '&' operator
ideal_weather_condition = temperature_condition & wind_speed_condition & cloudiness_condition

# Use the boolean mask to filter the DataFrame
ideal_weather_cities = city_data_df[ideal_weather_condition]

# Display sample data
ideal_weather_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
51,51,zhob,31.3411,69.4481,25.16,9,0,1.71,PK,1696164324
84,84,hawaiian paradise park,19.5933,-154.9731,22.80,94,0,3.09,US,1696163932
109,109,holualoa,19.6228,-155.9522,25.48,82,0,3.60,US,1696164353
122,122,villarrica,-25.7500,-56.4333,23.42,56,0,2.86,PY,1696164359
202,202,heilbron,-27.2812,27.9709,26.01,17,0,4.30,ZA,1696164396
258,258,chandler,33.3062,-111.8412,22.13,48,0,2.06,US,1696164409
300,300,hilo,19.7297,-155.0900,23.12,86,0,3.09,US,1696164069
327,327,hotan,37.0997,79.9269,21.37,22,0,1.16,CN,1696163929
406,406,hanceville,34.0607,-86.7675,21.42,68,0,1.67,US,1696164445
412,412,colonia,50.9333,6.9500,26.64,53,0,3.60,DE,1696164044


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

# Create a new DataFrame 'hotel_df' by copying selected columns from 'city_data_df'
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,nago,JP,26.5881,127.9761,83,
1,port-aux-francais,TF,-49.3500,70.2167,97,
2,jamestown,US,42.0970,-79.2353,100,
3,richards bay,ZA,-28.7830,32.0377,64,
4,punta arenas,CL,-53.1500,-70.9167,70,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "api_key": geoapify_key,
    "radius": radius,
    "categories" :"accommodation.hotel"
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    #print(name_address)
   
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        #hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df





Starting hotel search
nago - nearest hotel: グリーンリッチホテル
port-aux-francais - nearest hotel: Keravel
jamestown - nearest hotel: DoubleTree Jamestown
richards bay - nearest hotel: Bon Hotel Waterfront
punta arenas - nearest hotel: Hotel Lacolet
hadibu - nearest hotel: No hotel found
vadso - nearest hotel: Scandic Vadsø
waitangi - nearest hotel: Hotel Chathams
tazovsky - nearest hotel: Тазовчанка
coxen hole - nearest hotel: Elizabeth Inn
sao felix do xingu - nearest hotel: Hotel Terraço
grytviken - nearest hotel: No hotel found
marang - nearest hotel: Hotel Syahirah
lloydminster - nearest hotel: Central Suite Hotel Knight's Inn
la'ie - nearest hotel: No hotel found
georgetown - nearest hotel: Page 63 hostel
qaqortoq - nearest hotel: Hotel Qaqortoq
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by Wyndham
talnakh - nearest hotel: Талнах
tiksi - nearest hotel: Арктика
luau - nearest hotel: No hotel found
bredasdorp - nearest hotel: Victoria Hotel
greenville -

dzhambeyty - nearest hotel: Алаш
strovolos - nearest hotel: Europa Plaza Hotel
progreso - nearest hotel: Centro Vacacional Obrero CTM
neiafu - nearest hotel: Port Wine Guest House
palikir - national government center - nearest hotel: Mangrove Bay Hotel
puerto aysen - nearest hotel: Hostal Dendal
lashio - nearest hotel: Golden Kaniri Hotel
sao vicente - nearest hotel: Pousada Vitória
narrabri - nearest hotel: Tattersall's Hotel
gondar - nearest hotel: Hotel Dashen
tuy hoa - nearest hotel: Vinh Tuan Hotel
heilbron - nearest hotel: No hotel found
caluula - nearest hotel: No hotel found
waddan - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
rochester - nearest hotel: Holiday Inn Rochester Downtown
newbury - nearest hotel: The Elephant at the Market
port hedland - nearest hotel: The Esplanade Hotel
thompson - nearest hotel: Thompson Inn
ciudad constitucion - nearest hotel: Hotel El Arbolito
neelankarai - nearest hotel: Hotel Park Plaza Chennai OMR
fale old se

weno - nearest hotel: High Tide Hotel
ulsteinvik - nearest hotel: Quality Hotel Ulstein
odesskoye - nearest hotel: No hotel found
sile - nearest hotel: No hotel found
bend - nearest hotel: Old St. Francis School
sassandra - nearest hotel: Hotel Le Pollet
ubari - nearest hotel: فندق اوباري
asuncion nochixtlan - nearest hotel: Hotel Rincón de los 3 Ángeles
ternate - nearest hotel: Boulivard ternate
susuman - nearest hotel: No hotel found
delvada - nearest hotel: Pensao Beira Mar
tutong - nearest hotel: No hotel found
bonthe - nearest hotel: No hotel found
iskateley - nearest hotel: Авантаж
ponsacco - nearest hotel: Hotel La Pace
moose jaw - nearest hotel: Temple Gardens Mineral Spa Resort
maun - nearest hotel: Center Lodge Conference Center
troitskoye - nearest hotel: Березка
la spezia - nearest hotel: Hotel Ghironi
qarah bagh - nearest hotel: No hotel found
tarakan - nearest hotel: Hotel Dynasty
frontera - nearest hotel: Quirino
ulysses - nearest hotel: No hotel found
berdigestyakh - ne

kuqa - nearest hotel: Kuqa Hotel - foreigners accepted
kisiwani - nearest hotel: No hotel found
moa - nearest hotel: Hotel Miraflores
ghanzi - nearest hotel: Ghanzi farmhouse
aioun - nearest hotel: فندق العيون
eleskirt - nearest hotel: Öğretmen evi
siocon - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,nago,JP,26.5881,127.9761,83,グリーンリッチホテル
1,port-aux-francais,TF,-49.3500,70.2167,97,Keravel
2,jamestown,US,42.0970,-79.2353,100,DoubleTree Jamestown
3,richards bay,ZA,-28.7830,32.0377,64,Bon Hotel Waterfront
4,punta arenas,CL,-53.1500,-70.9167,70,Hotel Lacolet
...,...,...,...,...,...,...
567,moa,CU,20.6569,-74.9403,81,Hotel Miraflores
568,ghanzi,BW,-21.5667,21.7833,6,Ghanzi farmhouse
569,aioun,MR,16.6614,-9.6149,9,فندق العيون
570,eleskirt,TR,39.8033,42.6736,40,Öğretmen evi


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
scatter_plot = city_data_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Humidity',
    s='Humidity',
    cmap='Blues',
    colorbar=True,
    title='City Humidity Map',
    width=800,
    height=600,
    hover_cols=['Hotel Name', 'Country', 'Humidity (%)']  # Customize the hover tooltip
)

# Display the map
scatter_plot

:Scatter   [Lng]   (Lat,Humidity,Country)